# Explication des données à manipuler (valeur numérique, échelle  de 0 à 10)
fever – Intensité de la fièvre 

dry_cough – Intensité de la toux sèche

tiredness – Niveau de fatigue

aches_and_pains – Douleurs musculaires ou courbatures

sore_throat – Intensité du mal de gorge

diarrhoea – Sévérité de la diarrhée

conjunctivitis – Intensité de la conjonctivite

headache – Intensité du mal de tête

loss_of_taste – Perte du goût

difficulty_breathing – Difficulté à respirer

chest_pain – Douleur thoracique

rash_on_skin – Présence d’éruptions cutanées

SARS_Cov_2_exam_result – Résultat du test COVID-19 : "positive" ou "negative"

In [1]:
import tenseal as ts
import pandas as pd
import utils
import numpy as np

In [ ]:
covid_data = pd.read_csv('C:/Users/Lambika/Desktop/stage_CETIC/DATA_SET/covid19.csv')

columns = [
    "id","fever", "dry_cough", "tiredness", "aches_and_pains", "sore_throat",
    "diarrhoea", "conjunctivitis", "headache", "loss_of_taste",
    "difficulty_breathing", "chest_pain", "rash_on_skin", "SARS_Cov_2_exam_result"
]

for col in columns:
    globals()[col] = covid_data[col].tolist()
    
 
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=16384,
    coeff_mod_bit_sizes=[60, 40,40, 60]
)
context.global_scale = 2 ** 40
context.generate_galois_keys()
my_secret_key = context.serialize(save_secret_key=True)
utils.write_data("Keys_provider/secret_key.txt", my_secret_key)
context.make_context_public()
public_key=context.serialize()
utils.write_data("Keys_provider/public_key.txt", public_key)


#cryptage de données
id_encrypted = ts.ckks_vector(context, id)
fever_encrypted = ts.ckks_vector(context, fever)
dry_cough_encrypted = ts.ckks_vector(context, dry_cough)
tiredness_encrypted = ts.ckks_vector(context, tiredness)
aches_and_pains_encrypted = ts.ckks_vector(context, aches_and_pains)
sore_throat_encrypted = ts.ckks_vector(context, sore_throat)
diarrhoea_encrypted = ts.ckks_vector(context, diarrhoea)
conjunctivitis_encrypted = ts.ckks_vector(context, conjunctivitis)
headache_encrypted = ts.ckks_vector(context, headache)
loss_of_taste_encrypted = ts.ckks_vector(context, loss_of_taste)
difficulty_breathing_encrypted = ts.ckks_vector(context, difficulty_breathing)
chest_pain_encrypted = ts.ckks_vector(context, chest_pain)
for i in range(len(SARS_Cov_2_exam_result)):
    if SARS_Cov_2_exam_result[i] == "positive":
        SARS_Cov_2_exam_result[i] = 1
    else:
        SARS_Cov_2_exam_result[i] = 0   
print("sum of covid_positive cases: ", sum(SARS_Cov_2_exam_result))        
rash_on_skin_encrypted = ts.ckks_vector(context, rash_on_skin)

# print(SARS_Cov_2_exam_result)
SARS_Cov_2_exam_result_encrypted = ts.ckks_vector(context, SARS_Cov_2_exam_result)
# Encrypted data serialization
utils.write_data("Provider_Encrypted_data/id_encrypted.txt", id_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/fever_encrypted.txt", fever_encrypted.serialize())  
utils.write_data("Provider_Encrypted_data/dry_cough_encrypted.txt", dry_cough_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/tiredness_encrypted.txt", tiredness_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/aches_and_pains_encrypted.txt", aches_and_pains_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/sore_throat_encrypted.txt", sore_throat_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/diarrhoea_encrypted.txt", diarrhoea_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/conjunctivitis_encrypted.txt", conjunctivitis_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/headache_encrypted.txt", headache_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/loss_of_taste_encrypted.txt", loss_of_taste_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/difficulty_breathing_encrypted.txt", difficulty_breathing_encrypted.serialize())
utils.write_data("Providerncrypted_data/chest_pain_encrypted.txt", chest_pain_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/rash_on_skin_encrypted.txt", rash_on_skin_encrypted.serialize())
utils.write_data("Provider_Encrypted_data/SARS_Cov_2_exam_result_encrypted.txt", SARS_Cov_2_exam_result_encrypted.serialize())     


sum of covid_positive cases:  302


In [3]:
from flask import Flask, send_from_directory, render_template, request, redirect, url_for
import os
from werkzeug.utils import secure_filename

app = Flask(__name__)

# Configuration
UPLOAD_FOLDER = 'uploads'
ALLOWED_EXTENSIONS = {'txt'}
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Créer les dossiers nécessaires
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs('Keys_provider', exist_ok=True)
os.makedirs('Provider_Encrypted_data', exist_ok=True)

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/download/<filename>")
def download_file(filename):
    # Déterminer le bon dossier en fonction du fichier
    if filename == "public_key.txt":
        directory = os.path.join(os.getcwd(), "Keys_provider")
    else:
        directory = os.path.join(os.getcwd(), "Provider_Encrypted_data")
    
    try:
        return send_from_directory(directory, filename, as_attachment=True)
    except FileNotFoundError:
        # Retourner une réponse 404 avec un message personnalisé
        return f"Le fichier {filename} n'a pas été trouvé sur le serveur.", 404

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return redirect(request.url)
    
    files = request.files.getlist('file')
    
    for file in files:
        if file.filename == '':
            continue
            
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
    
    return redirect(url_for('home'))

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.7.238:5000
Press CTRL+C to quit
192.168.7.238 - - [28/May/2025 09:40:40] "GET / HTTP/1.1" 200 -
192.168.7.238 - - [28/May/2025 09:40:41] "GET /favicon.ico HTTP/1.1" 404 -
192.168.7.238 - - [28/May/2025 09:44:34] "GET /download/SARS_Cov_2_exam_result_encrypted.txt HTTP/1.1" 200 -
192.168.7.238 - - [28/May/2025 09:45:25] "GET /download/aches_and_pains_encrypted.txt HTTP/1.1" 200 -
192.168.7.238 - - [28/May/2025 09:45:35] "GET /download/fever_encrypted.txt HTTP/1.1" 200 -
192.168.7.238 - - [28/May/2025 09:45:47] "GET /download/dry_cough_encrypted.txt HTTP/1.1" 200 -
192.168.7.238 - - [28/May/2025 09:45:55] "GET /download/chest_pain_encrypted.txt HTTP/1.1" 200 -
192.168.7.238 - - [28/May/2025 09:46:39] "GET /download/public_key.txt HTTP/1.1" 200 -
192.168.7.238 - - [28/May/2025 09:49:58] "POST /upload HTTP/1.1" 302 -
192.168.7.238 - - [28/May/2025 09:49:58] "GET / HTTP/1.1" 200 -
192.168.

In [4]:
loaded_secret_key = utils.read_data("Keys_provider/secret_key.txt")
secret_key = ts.context_from(loaded_secret_key).secret_key()

nombre_de_cas_positive = utils.read_data("uploads/SARS_Cov_2_exam_result_SARS_Cov_2_exam_result_encrypted_computed.txt")
nombre_de_cas_positive = ts.lazy_ckks_vector_from(nombre_de_cas_positive)
nombre_de_cas_positive.link_context(context)
##dechiffrage
decrypted = nombre_de_cas_positive.decrypt(secret_key)
result = np.round(decrypted[0], decimals=0) 

print(f"Résultat brut (avec erreur CKKS): {decrypted[0]}")
print(f"Résultat corrigé: {int(result)}")  

score = utils.read_data("uploads/score_encrypted.txt")
score = ts.lazy_ckks_vector_from(score)
score.link_context(context)
decrypted_score =score.decrypt(secret_key)
print(f"Score brut (avec erreur CKKS): {decrypted_score}")
score = [a + b + c for a, b, c in zip(chest_pain, dry_cough, fever)]


print("real score brut", score)

Résultat brut (avec erreur CKKS): 302.00001072518336
Résultat corrigé: 302
Score brut (avec erreur CKKS): [18.103998651124975, 22.362120187078112, 24.24559862026539, 13.345894360123353, 13.262616640879719, 15.985409506411653, 19.89372474390414, 17.69803347755236, 12.729215979525815, 8.168504012698513, 16.353764896424792, 15.787944025896138, 22.627339674910516, 9.232674559800484, 17.474536525467137, 12.474344415781433, 11.094260399667542, 18.6788235755987, 19.435843894316513, 18.24350002111375, 10.180001137474623, 8.1017978071713, 26.13089428829584, 8.485003800813407, 15.239660958619043, 19.812296992032188, 11.093686780889893, 19.81229701391024, 11.093686808038461, 11.47589072870699, 7.551130189315947, 13.317716047582124, 23.610756017742816, 10.9830179989264, 16.968074564960958, 23.55239392066926, 17.268600965837138, 17.61974100598606, 9.803257956990011, 15.402778889593211, 9.050563239921473, 23.64761399629554, 22.070340606855027, 18.272779950606274, 8.168503972656074, 7.400980826080092